In [1]:
import numpy as np 
import pandas as pd 
from scipy import stats

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.



In [4]:
df = pd.read_table('water.txt')

df.head()

df.describe()

,mortality,hardness
count,61.000000,61.000000
mean,1524.147541,47.180328
std,187.668754,38.093966
min,1096.000000,5.000000
25%,1379.000000,14.000000
50%,1555.000000,39.000000
75%,1668.000000,75.000000
max,1987.000000,138.000000


In [11]:
print('Pearson corr:',round(stats.pearsonr(df['hardness'], df['mortality'])[0],4))

Pearson corr: -0.6548


В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки.



In [13]:
print('Spearman corr:',round(stats.spearmanr(df['mortality'],df['hardness'])[0],4))

Spearman corr: -0.6317


Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.



In [15]:
df.head(1)

,location,town,mortality,hardness
0,South,Bath,1247,105


In [18]:
print('Pearson South Cities corr:',round(stats.pearsonr(df[df['location']=='South']['hardness'],
                                           df[df['location']=='South']['mortality'])[0],4))

Pearson South Cities corr: -0.6022


In [22]:
print('Pearson North Cities corr:',round(abs(stats.pearsonr(df[df['location']=='North']['hardness'],
                                           df[df['location']=='North']['mortality'])[0]),4))

Pearson North Cities corr: 0.3686


Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

In [25]:
#для коэфф-та корр-ции Мэтьюса нет готовой реализации, беру формулу из конспекта
survey_data = np.array([[203., 239.], [718., 515.]])
survey_data

array([[203., 239.],
       [718., 515.]])

In [75]:
def Matthews(a,b,c,d):
    return (a*d - c*b)/np.sqrt((a+b)*(a+c)*(b+d)*(c+d))

In [76]:
print('Matthews correl coef:',round(Matthews(survey_data[0][0], survey_data[0][1], survey_data[1][0],survey_data[1][1]),3))

Matthews correl coef: -0.109


В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию $\textbf{scipy.stats.chi2_contingency}$. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$
 , нужно ввести 8).

In [44]:
print('p-value',stats.chi2_contingency(survey_data)[1])

p-value 1.0558987006638725e-05


В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.



In [45]:
#всё по-старинке --- берём готовую функцию из семинара (P.S. Спасибо, Эмили!)

def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)

    p1 = sample1[0] / np.sum(sample1)
    p2 = sample2[0] / np.sum(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ np.sum(sample1) + p2 * (1 - p2)/ np.sum(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ np.sum(sample1) + p2 * (1 - p2)/ np.sum(sample2))
    
    return (left_boundary, right_boundary)

In [77]:
print('interval [%.4f,%.4f]' %proportions_diff_confint_ind(survey_data[:,1], survey_data[:,0]))

interval [0.0539,0.1392]


Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$
 , нужно ввести 8).

In [50]:
#ah shit, here we go again 
#снова функции с семинара

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = np.sum(sample1)
    n2 = np.sum(sample2)
    
    p1 = sample1[0] / n1
    p2 = sample2[0] / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)

In [52]:
print('p-value: %f' % proportions_diff_z_test(proportions_diff_z_stat_ind(survey_data[:,1], survey_data[:,0])))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
float(proportions_diff_z_test(proportions_diff_z_stat_ind(survey_data[:,1], survey_data[:,0])))*10**6

8.153453089576601

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"

1. Не доволен	Более или менее	Доволен
2. Не очень счастлив	197	111	33
3. Достаточно счастлив	382	685	331
4. Очень счастлив	110	342	333

Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [59]:
happiness_data = np.array([[197., 111., 33.],[382., 685., 331.], [110., 342., 333.]])

In [63]:
print(round(stats.chi2_contingency(happiness_data)[0],4))

293.6831


На данных из предыдущего вопроса посчитайте значение достигаемого уровня значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$
 , нужно ввести 8).

In [64]:
print(stats.chi2_contingency(happiness_data)[1])

2.4964299580093467e-62


Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки.



In [72]:
def Kramer(array):
    return np.sqrt(stats.chi2_contingency(array)[0]/(array.sum()*(min(array.shape)-1)))

In [74]:
round(Kramer(happiness_data),4)

0.2412